<ul>
<li/>Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
<li/>Cохранить ее в pandas dataframe
<li/>Cохранить pandas dataframe в excel
<li/>Cохранить pandas dataframe в pickle
<li/>Cохранить pandas dataframe в БД
</ul>

In [1]:
import requests
import pandas as pd
from tqdm.auto import tqdm
from collections import defaultdict
import math
from bs4 import BeautifulSoup

In [2]:
limit_ = 100
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
res = requests.get(url, params={'objStatus': '0'})
num_pages = math.ceil(res.json().get('data').get('total')/limit_)

In [3]:
objids = []
offset_ = 0
for i in tqdm(range(num_pages)):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
    res = requests.get(url, params={'limit': limit_, 
                                    'offset': offset_, 
                                    'objStatus': '0', 
                                    'sortField': 'devId.devShortCleanNm',
                                    'sortType': 'asc'})
    offset_ += limit_
    objects_list = res.json().get('data').get('list')
    objids.extend([x.get('objId') for x in objects_list])

  0%|          | 0/108 [00:00<?, ?it/s]

In [4]:
dom_info = defaultdict(list)
for objid in tqdm(objids):
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objid}'
        img = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82/{objid}'
        parsed_html = BeautifulSoup(requests.get(img).content)
        src = [(link.get('src')) for link in parsed_html.body.find_all('img', attrs={'class':'styles__Preview-sy86zr-5 hQnKUU'})]
        res = requests.get(url)
        data = res.json().get('data')
        dom_info['id'].append(data.get('id'))
        dom_info['developer'].append('{} {}'.format(data.get('developer').get('orgForm').get('shortForm'), 
                                                    data.get('developer').get('devFullCleanNm')))
        dom_info['dom_region'].append(data.get('region'))
        dom_info['data_finish'].append(data.get('objReady100PercDt'))
        dom_info['num_of_floors'].append(data.get('objFloorCnt'))
        dom_info['image'].append(', '.join(src))
    except:
        continue

  0%|          | 0/10786 [00:00<?, ?it/s]

In [6]:
df = pd.DataFrame(dom_info)
df.head()

,id,developer,dom_region,data_finish,num_of_floors,image
0,13391,АО 2МЕН ГРУПП,72,2022-12-31,12,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
1,13383,АО 2МЕН ГРУПП,72,2022-12-31,20,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
2,13398,АО 2МЕН ГРУПП,72,2024-12-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
3,13397,АО 2МЕН ГРУПП,72,2024-12-31,12,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
4,13399,АО 2МЕН ГРУПП,72,2024-12-31,1,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...


In [11]:
df.data_finish = pd.to_datetime(df.data_finish)

In [25]:
import warnings

In [26]:
warnings.filterwarnings('ignore')

In [27]:
df.to_excel('nashdomrf.xlsx', index=False)

In [15]:
df.to_pickle('nashdomrf.pkl')

In [16]:
import sqlite3

In [17]:
conn = sqlite3.connect('nashdomrf_database')

In [18]:
df.to_sql('under_construction', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

In [20]:
conn = sqlite3.connect('nashdomrf_database')
pd.read_sql('select * from under_construction limit 10', conn)

,id,developer,dom_region,data_finish,num_of_floors,image
0,13391,АО 2МЕН ГРУПП,72,2022-12-31 00:00:00,12,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
1,13383,АО 2МЕН ГРУПП,72,2022-12-31 00:00:00,20,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
2,13398,АО 2МЕН ГРУПП,72,2024-12-31 00:00:00,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
3,13397,АО 2МЕН ГРУПП,72,2024-12-31 00:00:00,12,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
4,13399,АО 2МЕН ГРУПП,72,2024-12-31 00:00:00,1,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
5,36912,ООО CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК СТРОЙРЕСУРС,26,2022-12-31 00:00:00,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
6,10891,АО АКЦИОНЕРНАЯ АГРОСТРОИТЕЛЬНАЯ КОМПАНИЯВОЛОГД...,35,2023-03-31 00:00:00,16,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
7,8809,ООО АБСОЛЮТ,74,2019-03-31 00:00:00,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
8,8810,ООО АБСОЛЮТ,74,2019-03-31 00:00:00,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
9,39176,ООО АВА ДЕВЕЛОПМЕНТ ПЛЮС,23,2024-09-30 00:00:00,25,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...


In [21]:
conn.close()

In [23]:
pd.read_excel('nashdomrf.xlsx').head(20)

,id,developer,dom_region,data_finish,num_of_floors,image
0,13391,АО 2МЕН ГРУПП,72,2022-12-31,12,NaN
1,13383,АО 2МЕН ГРУПП,72,2022-12-31,20,NaN
2,13398,АО 2МЕН ГРУПП,72,2024-12-31,10,NaN
3,13397,АО 2МЕН ГРУПП,72,2024-12-31,12,NaN
4,13399,АО 2МЕН ГРУПП,72,2024-12-31,1,NaN
5,36912,ООО CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК СТРОЙРЕСУРС,26,2022-12-31,10,NaN
6,10891,АО АКЦИОНЕРНАЯ АГРОСТРОИТЕЛЬНАЯ КОМПАНИЯВОЛОГД...,35,2023-03-31,16,NaN
7,8809,ООО АБСОЛЮТ,74,2019-03-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
8,8810,ООО АБСОЛЮТ,74,2019-03-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
9,39176,ООО АВА ДЕВЕЛОПМЕНТ ПЛЮС,23,2024-09-30,25,NaN


In [24]:
pd.read_pickle('nashdomrf.pkl').head(10)

,id,developer,dom_region,data_finish,num_of_floors,image
0,13391,АО 2МЕН ГРУПП,72,2022-12-31,12,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
1,13383,АО 2МЕН ГРУПП,72,2022-12-31,20,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
2,13398,АО 2МЕН ГРУПП,72,2024-12-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
3,13397,АО 2МЕН ГРУПП,72,2024-12-31,12,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
4,13399,АО 2МЕН ГРУПП,72,2024-12-31,1,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
5,36912,ООО CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК СТРОЙРЕСУРС,26,2022-12-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
6,10891,АО АКЦИОНЕРНАЯ АГРОСТРОИТЕЛЬНАЯ КОМПАНИЯВОЛОГД...,35,2023-03-31,16,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
7,8809,ООО АБСОЛЮТ,74,2019-03-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
8,8810,ООО АБСОЛЮТ,74,2019-03-31,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
9,39176,ООО АВА ДЕВЕЛОПМЕНТ ПЛЮС,23,2024-09-30,25,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...
